Chapter 1

Solving the problem:

maximize profit:<br>
25 * XB + 30 * XC

st:<br>
200 * XB <= 6000<br>
140 * XC <= 4000<br>
(XB / 200) + (XC / 140) <= 40

In [2]:
%pip install -q amplpy

Note: you may need to restart the kernel to use updated packages.


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^


In [22]:
from amplpy import AMPL
ampl = AMPL()
ampl.option['solver'] = 'highs'

Declare model and data

In [32]:
# must be reset to be able to re-run
ampl.reset()

ampl.eval(
'''
set PROD;
set STAGE;

param rate {PROD, STAGE} > 0;
param avail {STAGE} >= 0;
param profit {PROD} >= 0;
param commit {PROD} >= 0;
param market {PROD} >= 0;

var Make {p in PROD} >= commit[p], <= market[p];
maximize Total_Profit: sum {p in PROD} profit[p] * Make[p];
subject to Time {s in STAGE}: sum {p in PROD} (1/rate[p, s]) * Make[p] <= avail[s];
''')

ampl.set['PROD'] = ['bands', 'coils', 'plate']
ampl.set['STAGE'] = ['reheat', 'roll']
ampl.param['rate'] = {
    ('bands', 'reheat'): 200, ('bands', 'roll'): 200,
    ('coils', 'reheat'): 200, ('coils', 'roll'): 140,
    ('plate', 'reheat'): 200, ('plate', 'roll'): 160
    }
ampl.param['profit'] = [25, 30, 29]
ampl.param['commit'] = [1000, 500, 750]
ampl.param['market'] = [6000, 4000, 3500]
ampl.param['avail'] = [35, 40]

ampl.eval('''expand;''')

ampl.solve()

maximize Total_Profit:
	25*Make['bands'] + 30*Make['coils'] + 29*Make['plate'];

subject to Time['reheat']:
	0.005*Make['bands'] + 0.005*Make['coils'] + 0.005*Make['plate'] <= 35;

subject to Time['roll']:
	0.005*Make['bands'] + 0.00714286*Make['coils'] + 0.00625*Make['plate']
	 <= 40;

HiGHS 1.7.1:HiGHS 1.7.1: optimal solution; objective 190071.4286
2 simplex iterations
0 barrier iterations


In [37]:
ampl.display('Total_Profit')
ampl.display('Make')

Total_Profit = 190071

Make [*] :=
bands  3357.14
coils   500
plate  3142.86
;



Display marginal values

In [38]:
ampl.display('Time')
ampl.display('Make.lb')
ampl.display('Make.ub')
ampl.display('Make.rc')

Time [*] :=
reheat  1800
  roll  3200
;

Make.lb [*] :=
bands  1000
coils   500
plate   750
;

Make.ub [*] :=
bands  6000
coils  4000
plate  3500
;

Make.rc [*] :=
bands  -5.32907e-15
coils  -1.85714
plate  -7.10543e-15
;

